### First Script - Frequency Compiling and Export to CSV

This script was created on 2022-10-06. It can perform frequency compiling based a directory that contains XML files, returned by ProQuest TMD Studio search.  \
It has two required file path that needed to be inputted manually. The first one is located at cell 4 for the input data directory and the second one is located at cell 19, which is the directory of the frequency count csv file.

In [1]:
# Libraries for parsing data
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import os

# Library for plotting data
import matplotlib.pyplot as plt

In [2]:
str1= '20230826sample308'

In [3]:
# Set corpus to the folder of files you want to use
corpus = '/home/ec2-user/SageMaker/data/' + str1 + '/'

# Read in files
input_files = os.listdir(corpus)

In [4]:
# Function to strip html tags from text portion
def strip_html_tags(text):
    stripped = BeautifulSoup(text).get_text().replace('\n', ' ').replace('\\', '').strip()
    return stripped

In [5]:
def getxmlcontent(corpus, file, strip_html=True):
    try:
        tree = etree.parse(corpus + file)
        root = tree.getroot()

        if root.find('.//GOID') is not None:
            goid = root.find('.//GOID').text
        else:
            goid = None

        if root.find('.//Title') is not None:
            title = root.find('.//Title').text
        else:
            title = None

        if root.find('.//NumericDate') is not None:
            date = root.find('.//NumericDate').text
        else:
            date = None
            
        if root.find('.//PublisherName') is not None:
            publisher = root.find('.//PublisherName').text
        else:
            publisher = None

        if root.find('.//FullText') is not None:
            text = root.find('.//FullText').text

        elif root.find('.//HiddenText') is not None:
            text = root.find('.//HiddenText').text

        elif root.find('.//Text') is not None:
            text = root.find('.//Text').text

        else:
            text = None

        # Strip html from text portion
        if text is not None and strip_html == True:
            text = strip_html_tags(text)
    
    except Exception as e:
        print(f"Error while parsing file {file}: {e}")
    
    return goid, title, date, publisher, text

In [6]:
# Columns lists
goid_list = []
publisher_list = []
title_list = []
text_list = []
date_list = []

# Used for grouping by publisher
publishers = []

for file in input_files:
    try:
        goid, title, date, publisher, text = getxmlcontent(corpus, file, strip_html=True)

        goid_list.append(goid)
        publisher_list.append(publisher)
        title_list.append(title)
        text_list.append(text)
        date_list.append(date)
    except:
        pass

Error while parsing file metadata: Document is empty, line 1, column 1 (metadata, line 1)


In [7]:
# Transform processed data into a dataframe
df = pd.DataFrame({'GOID': goid_list, 'Publisher': publisher_list, 'Title':title_list, 'Text': text_list, 'Date': date_list})

In [8]:
len(df)

5909

In [9]:
df = df.dropna()

In [10]:
random_df = df.sample(n=200, random_state=123)

In [11]:
random_df.to_csv('/home/ec2-user/SageMaker/data/20230828/' + str1 + 'NYT_articles.csv')

In [12]:
#df.sort_values(by='Date', inplace=True)

In [13]:
# Convert date column into type datetime
df['Date'] = pd.to_datetime(df['Date'])

# Remove column "Date" without value
#df = df.dropna(subset=['Date'], )

In [14]:
# Format the table to frequency table 
new_df = df.groupby([df['Date'].dt.year.rename('year'), df['Date'].dt.month.rename('month')]).agg({'count'}, dropna=False)

In [15]:
# Drop "Publisher", "Text" and "Date"
new_df = new_df.drop(columns=['GOID','Publisher', 'Text', 'Date'])

In [16]:
#change name of dataframe
new_df.columns =['count']

In [17]:
#reset as dataframe from multi-index
#new_df = new_df.reset_index()

In [18]:
#month_df = pd.read_csv('/home/ec2-user/SageMaker/data/NYT_scores/2023-07-18-fueleconomy-index_monthly.csv')
#quarter_df = pd.read_csv('/home/ec2-user/SageMaker/data/NYT_scores/2023-07-18-fueleconomy-index_quarter.csv')

In [19]:
month_df = pd.read_csv('/home/ec2-user/SageMaker/data/20230824sample101month.csv')
quarter_df = pd.read_csv('/home/ec2-user/SageMaker/data/20230824sample101quarter.csv')

In [20]:
monthly_df = new_df.merge(month_df, on = ['year', 'month'], how = 'right')

In [21]:
#monthly_df = new_df.merge(month_df, left_on=['year','month'], right_on = ['year','month'], how='right')

In [22]:
monthly_df = monthly_df.fillna(0)

In [23]:
monthly_df['normalized'] = monthly_df['count']/month_df['nb all articles']
monthly_df['nb relevant articles'] = monthly_df['count']
monthly_df = monthly_df.drop(columns = ['count'])

In [24]:
#monthly_df['nb all articles'] = monthly_df['score']
#monthly_df['nb relevant articles'] = monthly_df['count']
#monthly_df = monthly_df[['normalized', 'year','month', 'nb all articles', 'nb relevant articles']]

In [25]:
monthly_df.to_csv('/home/ec2-user/SageMaker/data/20230828/' + str1 + 'month.csv', index=False)

In [26]:
new_df = df.groupby([df['Date'].dt.year.rename('year'), df['Date'].dt.quarter.rename('quarter')]).agg({'count'}, dropna=False)
new_df = new_df.drop(columns=['GOID','Publisher', 'Text', 'Date'])
new_df.columns =['count']
new_df = new_df.reset_index()

In [27]:
quarterly_df = new_df.merge(quarter_df, on=['year','quarter'], how='right')
quarterly_df = quarterly_df.fillna(0)
quarterly_df['normalized'] = quarterly_df['count']/month_df['nb all articles']
quarterly_df['nb relevant articles'] = quarterly_df['count']
quarterly_df = quarterly_df.drop(columns = ['count'])
quarterly_df = quarterly_df[['normalized', 'year','quarter', 'nb all articles', 'nb relevant articles']]

In [28]:
quarterly_df.to_csv('/home/ec2-user/SageMaker/data/20230828/'+str1+'quarter.csv', index=False)